In [45]:
import networkx as nx
# construct the sample graph with the networkx library.
# This custom DAG represents a recipe instruction workflow.

graph = nx.DiGraph()
graph.add_edges_from([
    ("Preheat[0]", "Chop[1]"),
    ("Preheat[0]", "Whisk[2]"),
    ("Chop[1]", "Bake[3]"),
    ("Whisk[2]", "Bake[3]"),
    ("Chop[1]", "Cool[4]"),
    ("Bake[3]", "Cool[4]"),
    ("Bake[3]", "Garnish[5]")
])


In [46]:
# infer the nodes from the  collection of edges above.
graph.nodes()   

NodeView(('Preheat[0]', 'Chop[1]', 'Whisk[2]', 'Bake[3]', 'Cool[4]', 'Garnish[5]'))

### Check validity for Topological Sorting


In [47]:
    
### Check validity for Topological Sorting
nx.is_directed(graph) # => True
# Since it's a DAG, topological sort can be conducted

True

In [48]:
  # Since it's a DAG, topological sort can be conducted
nx.is_directed_acyclic_graph(graph) # => True

True

In [49]:
# If it wasn's a DAG, this would throw an error
list(nx.topological_sort(graph))

['Preheat[0]', 'Chop[1]', 'Whisk[2]', 'Bake[3]', 'Cool[4]', 'Garnish[5]']

In [63]:
from collections import defaultdict, deque
import time
import networkx as nx

import random

# defaultdict provides a default value for non-existent keys 
# builds an adjacency list for the graph


#Class to represent a graph
class Graph:
    def __init__(self,vertices):# my graph
        self.graph = defaultdict(list) # store adjacency List
        self.V = vertices #number of vertices

    # In adjascency list, add DIRECTED edge from node u to node v
    def addEdge(self,u,v):
        self.graph[u].append(v)

    # A recursive function used by topological Sort
    def DFStopologicalSortUtil(self,v,visited,stack):

        # Mark the current node as visited, eg 0 for 1st iteration so you don't revisit it.
        visited[v] = True

        # for each neighboring node i of node 0 for example, if not visited, recursively call topological sort on i
        for i in self.graph[v]:
            if visited[i] == False:
                self.DFStopologicalSortUtil(i,visited,stack)

        # After visiting all neighbors of v, insert v at the front of the stack.
        # nodes with no outgoing edges go last. 
        stack.insert(0,v)

      # The function to do Topological Sort. It uses the recursive DFStopologicalSortUtil()
    
    def dfs_topologicalSort(self):
        # Mark all the vertices as not visited
        visited = [False]*self.V
        stack =[]# Will contain topologically sorted nodes

        # for each node from 0 to n-1, call the DFS algorithm
        for i in range(self.V):
            if visited[i] == False:
                self.DFStopologicalSortUtil(i,visited,stack)
         # Print contents of stack
        return stack

     # Kahn’s algorithm
    def Kahn_topological_sort(self):
        in_degree = defaultdict(int)# Keep track of how many incoming edges each vertex has
        # setting indegree value for each vertices

        for u in self.graph:# loop through all edges in the graph
            for v in self.graph[u]:
                in_degree[v] += 1# Increment

         # create queue
        queue = deque()
        
        # keep track of all nodes with in-degree 0(no incoming edges)
        for u in self.graph:
            if in_degree[u] == 0:
                queue.append(u)

        result = []# to store final topological ordering
        while queue:
            u = queue.popleft()# Remove a node from the front of the queue
            result.append(u)# Add it to the topological order
            
            
            for v in self.graph[u]:
                in_degree[v] -= 1# Reduce the degree by 1 since on of the dependancies has been removed
                if in_degree[v] == 0:# Add node to queue since it has no incoming nodes
                    queue.append(v)
        
        return result
     
    def heuristic_topo_sort(self):
        in_deg = defaultdict(int)# number of incoming edges
        out_deg = defaultdict(int)# number of outgoing edges

        for u in range(self.V):# for each node u 
            for v in self.graph[u]:# a node with no outgoing edges will not enter this loop
                out_deg[u] =out_deg[u]+ 1# increment because u has an outgoing edge to v
                in_deg[v] =in_deg[v] + 1# increment because v has an incoming edge from u

        scores = []

        for node in range(self.V): # compute the score for each node
            diff = out_deg[node] - in_deg[node]  
            scores.append((node, diff)) # stored as a tuple

        scores.sort(key=lambda x: -x[1])  # we sort the scores in descending order such that highest score comes first

        sorted_nodes = []
        for node, score in scores:
            sorted_nodes.append(node)# Extract the node order from the sorted list

        # This is the ordered list of nodes
        return sorted_nodes


def create_random_directed_graph(num_nodes, edge_probability):
# This function creates a random directed graph.

# with parameters as # of nodes, num_nodes: The number of nodes
# and edge_probability being the probability of an edge existing between any two nodes.

    while True:
        graph_nx = nx.fast_gnp_random_graph(num_nodes, edge_probability, directed=True)
        if nx.is_directed_acyclic_graph(graph_nx):
            break

    g = Graph(num_nodes)
    for u, v in graph_nx.edges():
        g.addEdge(u, v)
    return g
# returns a directed acyclic graph


    
# Function to evaluate sorting performance
def test_performance():
    sizes = [5, 10, 14, 32]  # Different input sizes
    edge_probability = 0.1
    results = []
    
    total_tests = 0
    heuristic_matches = 0

    for size in sizes:
        g = create_random_directed_graph(size, edge_probability)

        
        # Run all of the 3  algorithms
        dfs_output = g.dfs_topologicalSort()
        kahn_output  = g.Kahn_topological_sort()
        heuristic_output  = g.heuristic_topo_sort()

        # Compare heuristic with the kahn’s output
        if heuristic_output  == kahn_output:
            heuristic_matches += 1
        total_tests += 1
        
        
        
        start_dfs = time.time()
        g.dfs_topologicalSort()
        end_dfs = time.time()
        #dfstime.append((end_dfs - start_dfs))

        start_kahn = time.time()
        g.Kahn_topological_sort()
        end_kahn = time.time()
        #kahntime.append((end_kahn - start_kahn))
        
        start_heuristic = time.time()
        g.heuristic_topo_sort()
        end_heuristic = time.time()

        
        results.append({
            "nodes": size,
            "dfs_time": end_dfs - start_dfs,
            "kahn_time": end_kahn - start_kahn,
            "heuristic_time": end_heuristic - start_heuristic
        })

    heuristic_accuracy = (heuristic_matches / total_tests) * 100
    print(f"my heuristic algorithm matched the kahn algorithm in {heuristic_accuracy:.2f} percent of cases.")
    
    return results

    
if __name__ == '__main__':
    # This is a manual test case
    g = Graph(6)
    g.addEdge(0, 1)
    g.addEdge(0, 2)
    g.addEdge(1, 3)
    g.addEdge(2, 3)
    g.addEdge(3, 4)
    g.addEdge(3, 5)
    g.addEdge(1, 4)
    g.addEdge(2, 5)
    

    print("DFS-based Topological Sort:", g.dfs_topologicalSort())
    print("Kahn’s Topological Sort:", g.Kahn_topological_sort())
    print("Heuristic Topological Sort:", g.heuristic_topo_sort())


    # Performance test
    test_results = test_performance()
    for result in test_results:
        print(f"{result['nodes']} nodes — DFS: {result['dfs_time']:.7f}s, Kahn: {result['kahn_time']:.6f}s, Heuristic: {result['heuristic_time']:.7f}s")

DFS-based Topological Sort: [0, 2, 1, 3, 5, 4]
Kahn’s Topological Sort: [0, 1, 2, 3, 4, 5]
Heuristic Topological Sort: [0, 1, 2, 3, 4, 5]
my heuristic algorithm matched the kahn algorithm in 0.00 percent of cases.
5 nodes — DFS: 0.000000s, Kahn: 0.000000s, Heuristic: 0.000000s
10 nodes — DFS: 0.000000s, Kahn: 0.000000s, Heuristic: 0.000000s
14 nodes — DFS: 0.000000s, Kahn: 0.000997s, Heuristic: 0.000000s
32 nodes — DFS: 0.000000s, Kahn: 0.000000s, Heuristic: 0.000000s
